In [0]:
# -*- coding: utf-8 -*-
import dataiku
from mistralai import Mistral
from mistralai import DocumentURLChunk, ImageURLChunk, TextChunk
import tempfile
import base64
from io import BytesIO

client = dataiku.api_client()
project = client.get_default_project()
auth_info = client.get_auth_info(with_secrets=True)
MISTRAL_API_KEY = None
for secret in auth_info["secrets"]:
    if secret["key"] == "MISTRAL_API_KEY":
        MISTRAL_API_KEY = secret["value"]

client = Mistral(api_key=MISTRAL_API_KEY)

In [0]:
def extract_md_and_images(json_file_name, response_dict):
    base_name = os.path.splitext(json_file_name)[0]
    md_file_name = base_name + ".md"
    
    # Dictionnaire pour stocker les descriptions d'images
    image_descriptions = {}
    
    # Extraire, écrire les images et générer leurs descriptions
    has_images = False
    for page in response_dict.get("pages", []):
        if page.get("images", []):
            has_images = True
        for image in page.get("images", []):
            # Nom de fichier pour l'image
            image_file_name = base_name + "-" + image["id"]
            image_id = image["id"]  # ID utilisé dans le markdown pour référencer l'image
            
            # Écrire l'image si elle n'existe pas déjà
            if image_file_name not in existing_files:
                image_data = image["image_base64"]
                image_bytes = base64.b64decode(image_data.split(",")[1])
                with A220_tech_docs_prep.get_writer(image_file_name) as writer:
                    writer.write(BytesIO(image_bytes).getvalue())
            
            # Générer et écrire la description de l'image
            # Utiliser le format base_name-img-id.md pour la description
            description_file_name = base_name + "-" + image_id.split('.')[0] + ".md"
            if description_file_name not in existing_files:
                # Générer la description avec Mistral Vision
                description = generate_image_description(image["image_base64"])
                
                # Vérifier si la génération a échoué
                if description is None:
                    print(f"Échec lors de la génération de description pour l'image {image_id}")
                    return
                
                # Écrire la description dans un fichier .md
                with A220_tech_docs_prep.get_writer(description_file_name) as writer:
                    writer.write(description.encode('utf-8'))
                
                print(f"Description générée pour {image_id} et sauvegardée dans {description_file_name}")
            else:
                # Lire la description existante
                with A220_tech_docs_prep.get_download_stream(description_file_name) as reader:
                    description = reader.read().decode('utf-8')
            
            # Stocker la description pour mise à jour du markdown
            image_descriptions[image_id] = clean_description_for_alt_text(description)
    
    # Vérifier s'il y a des images dans le document
    if not has_images:
        print(f"Aucune image trouvée dans {json_file_name}, pas de création de fichier avec descriptions.")
        
        # Extraire et écrire le Markdown original sans modifications
        if md_file_name not in existing_files:
            markdown_content = "\n\n".join(page["markdown"] for page in response_dict.get("pages", []))
            with A220_tech_docs_prep.get_writer(md_file_name) as writer:
                writer.write(markdown_content.encode('utf-8'))
        return
    
    # Debug: Voir quelles images ont été traitées
    print(f"Images trouvées et décrites: {list(image_descriptions.keys())}")
    
    # Vérifier s'il y a des images et des descriptions dans le document
    if not image_descriptions:
        print(f"Aucune image trouvée ou décrite dans {json_file_name}, pas de création de fichier avec descriptions.")
        return
        
    # Créer un nouveau fichier markdown avec les descriptions d'images
    new_md_file_name = base_name + "__with_img_desc.md"
    
    # Vérifier si le fichier markdown avec descriptions existe déjà
    if new_md_file_name in existing_files:
        print(f"Le fichier {new_md_file_name} existe déjà, pas de régénération.")
        return
    
    # Vérifier si le fichier markdown original existe
    if md_file_name in existing_files:
        # Lire le contenu du markdown original
        with A220_tech_docs_prep.get_download_stream(md_file_name) as reader:
            markdown_content = reader.read().decode('utf-8')
    else:
        # Générer le markdown à partir des données de réponse
        markdown_content = "\n\n".join(page["markdown"] for page in response_dict.get("pages", []))
        # Écrire le markdown original s'il n'existe pas encore
        with A220_tech_docs_prep.get_writer(md_file_name) as writer:
            writer.write(markdown_content.encode('utf-8'))
    
    # Debug: afficher un extrait du markdown avant modification
    print(f"Extrait du markdown avant modification: {markdown_content[:200]}...")
    
    # Mettre à jour les références d'images avec les textes alternatifs
    updated_markdown = markdown_content
    replacements_made = False
    
    for image_id, description in image_descriptions.items():
        # Debug: Vérifier le pattern recherché
        pure_id = image_id.split('.')[0]  # Si image_id est "img-0.jpeg", prendre juste "img-0"
        print(f"Recherche de références pour l'image {image_id} (id sans extension: {pure_id})")
        
        # Chercher le pattern d'image sans texte alternatif ou avec texte alternatif vide
        pattern1 = rf'!\[\]?\(#image-{pure_id}\)'
        pattern2 = rf'!\[\]?\(#image-{image_id}\)'
        replacement = f'![{description}](#image-{image_id})'
        
        # Appliquer les remplacements et compter
        new_text, count1 = re.subn(pattern1, replacement, updated_markdown)
        if count1 > 0:
            updated_markdown = new_text
            replacements_made = True
            print(f"Remplacé {count1} occurrences du pattern {pattern1}")
        
        new_text, count2 = re.subn(pattern2, replacement, updated_markdown)
        if count2 > 0:
            updated_markdown = new_text
            replacements_made = True
            print(f"Remplacé {count2} occurrences du pattern {pattern2}")
        
        # Chercher également avec des textes alternatifs existants
        pattern3 = rf'!\[([^\]]*)\]\(#image-{pure_id}\)'
        pattern4 = rf'!\[([^\]]*)\]\(#image-{image_id}\)'
        
        new_text, count3 = re.subn(pattern3, replacement, updated_markdown)
        if count3 > 0:
            updated_markdown = new_text
            replacements_made = True
            print(f"Remplacé {count3} occurrences du pattern {pattern3}")
        
        new_text, count4 = re.subn(pattern4, replacement, updated_markdown)
        if count4 > 0:
            updated_markdown = new_text
            replacements_made = True
            print(f"Remplacé {count4} occurrences du pattern {pattern4}")
    
    # Vérifier si des remplacements ont été effectués
    if not replacements_made:
        print(f"Aucune référence d'image trouvée dans le markdown pour les remplacer. Vérifiez le format des références.")
        # Rechercher toutes les références d'images dans le markdown pour le débogage
        all_img_refs = re.findall(r'!\[.*?\]\(.*?\)', markdown_content)
        if all_img_refs:
            print(f"Références d'images trouvées dans le markdown: {all_img_refs[:5]}")
        else:
            print("Aucune référence d'image trouvée dans le document markdown.")
    
    # Écrire la version mise à jour du markdown dans un nouveau fichier quand même
    with A220_tech_docs_prep.get_writer(new_md_file_name) as writer:
        writer.write(updated_markdown.encode('utf-8'))
        print(f"Fichier {new_md_file_name} créé avec les descriptions d'images (remplacements effectués: {replacements_made}).")